In [ ]:
import os
import re
import sys
import glob
import pickle
from collections import OrderedDict

import numpy as np
from scipy.signal import welch#, periodogram
import pandas as pd
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Times New Roman']
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow import keras

if '..' not in sys.path:
    sys.path.append('..')
from dlml.utils import collect_experiments
from dlml.data import read_area_values, load_data_slide
from dlml.nn import predict

#### Find the best experiment given a set of tags

In [ ]:
area_IDs = [1, 2, 3]
area_measure = 'momentum'
H_G1 = 500
stoch_load_bus_IDs = [3]
rec_bus_IDs = [3, 14, 17]
rec_bus_IDs = []
D = 2
DZA = 0

experiments = collect_experiments(area_IDs, area_measure=area_measure, D=D, DZA=DZA, \
                                  stoch_load_bus_IDs=stoch_load_bus_IDs, H_G1=H_G1, \
                                  rec_bus_IDs=rec_bus_IDs, additional_tags=['ReLU_none'], \
                                  verbose=True)
if experiments is None:
    raise Exception('No experiment matches the tags')
experiment_IDs = list(experiments.keys())
experiment_ID = experiment_IDs[np.argmin([expt['val_loss'].min() for expt in experiments.values()])]
MAPE = experiments[experiment_ID]['MAPE']
if np.isscalar(MAPE):
    MAPE_str = f'{MAPE:.3f}%'
else:
    MAPE_str = '[' + '%, '.join([f'{m:.3f}' for m in MAPE]) + '%]'
loss = experiments[experiment_ID]['loss']
val_loss = experiments[experiment_ID]['val_loss']
batch_loss = experiments[experiment_ID]['batch_loss']
tags = experiments[experiment_ID]['tags']
print(f'The best experiment is {experiment_ID[:6]} (val_loss = {val_loss.min():.4f}, MAPE = {MAPE_str}).')

In [ ]:
offsets = {'77bf2ad6d2ff4986bbf9e9ee547bd0c1': 26}

In [ ]:
experiments_path = '../experiments/neural_network/'
checkpoint_path = experiments_path + experiment_ID + '/checkpoints/'
checkpoint_files = glob.glob(checkpoint_path + '*.h5')
network_parameters = pickle.load(open(experiments_path + experiment_ID \
                                      + '/parameters.pkl', 'rb'))
epochs = [int(os.path.split(file)[-1].split('.')[1].split('-')[0]) for file in checkpoint_files]
idx = np.argmin(val_loss) + 1
if experiment_ID in offsets:
    idx += offsets[experiment_ID]
best_checkpoint = checkpoint_files[epochs.index(idx)]
model = keras.models.load_model(best_checkpoint, compile=True)
data_dirs = ['..' + os.path.sep +
             os.path.sep.join([d for d in data_dir.split(os.path.sep) if '{}' not in d]) +
             f'/H_G1_{H_G1}/stoch_load_bus_' + '-'.join(map(str, stoch_load_bus_IDs))
             for data_dir in network_parameters['data_dirs']]
# we need mean and standard deviation of the training set to normalize the data
x_train_mean = network_parameters['x_train_mean']
x_train_std  = network_parameters['x_train_std']
ss = data_dirs[0].split(os.path.sep)
data_dir = os.path.join(os.path.join(*ss[:3]), 'Hiskens', os.path.join(*ss[3:]))
tmp = [re.findall('.*_bus', var_name)[0] for var_name in network_parameters['var_names']]
var_names_fmt = list(OrderedDict({k + '{}': [] for k in tmp}).keys())
if len(rec_bus_IDs) == 0:
    rec_bus_IDs = list(np.unique([int(re.findall('\d+', var_name)[0]) \
                                  for var_name in network_parameters['var_names']]))
    rec_bus_list = 'buses_' + '-'.join(map(str, rec_bus_IDs))
if not os.path.isdir(data_dir):
    raise Exception(f'{data_dir}: no such directory')
print(f'Loaded network from {best_checkpoint}.')
print(f'Data directory is {data_dir}.')
print(f'Variable names: {var_names_fmt}')

In [ ]:
keras.utils.plot_model(model, show_shapes=False, dpi=96)

In [ ]:
default_H = OrderedDict([
    ('Pg30', H_G1), ('Pg31', 30.3), ('Pg32', 35.8), ('Pg33', 28.6), ('Pg34', 26),
    ('Pg35', 34.8), ('Pg36', 26.4), ('Pg37', 24.3), ('Pg38', 34.5), ('Pg39', 42)
])

generators_areas_map = {
    'default': [
        ['Pg31', 'Pg32', 'Pg39'],
        ['Pg33', 'Pg34', 'Pg35', 'Pg36'],
        ['Pg37', 'Pg38'],
        ['Pg30']
    ],
    'compensator': [
        ['Pg31', 'Pg32', 'Pg39', 'Pg99'],
        ['Pg33', 'Pg34', 'Pg35', 'Pg36'],
        ['Pg37', 'Pg38'],
        ['Pg30']
    ]
}

P_nom = {gen_ID: 100e6 for gen_ID in default_H}

window_dur = 60
window_step = 1

var_names = [var_name.format(bus_ID) for bus_ID in np.unique(rec_bus_IDs) for var_name in var_names_fmt]
data_mean = {var_name: x_train_mean[k] for k,var_name in enumerate(var_names)}
data_std = {var_name: x_train_std[k] for k,var_name in enumerate(var_names)}

if area_measure == 'inertia':
    measure_units = 's'
elif area_measure == 'energy':
    measure_units = r'GW$\cdot$s'
elif area_measure == 'momentum':
    measure_units = r'GW$\cdot$s$^2$'
    
stoch_load_bus_list = 'stoch_load_bus_' + '-'.join(map(str, stoch_load_bus_IDs))
rec_bus_list = 'buses_' + '-'.join(map(str, rec_bus_IDs))

abbrv = {'inertia': 'H', 'energy': 'E', 'momentum': 'M'}

figure_width, figure_height = 8.5, 6

## Constant measure

In [ ]:
H_values = [
    default_H,
    OrderedDict([
        ('Pg30', H_G1), ('Pg31', 35), ('Pg32', 42), ('Pg33', 28.6), ('Pg34', 26),
        ('Pg35', 34.8), ('Pg36', 26.4), ('Pg37', 24.3), ('Pg38', 34.5), ('Pg39', 46)
    ]),
    OrderedDict([
        ('Pg30', H_G1), ('Pg31', 30.3), ('Pg32', 35.8), ('Pg33', 32), ('Pg34', 30),
        ('Pg35', 39), ('Pg36', 30), ('Pg37', 24.3), ('Pg38', 34.5), ('Pg39', 42)
    ]),
    OrderedDict([
        ('Pg30', H_G1), ('Pg31', 30.3), ('Pg32', 35.8), ('Pg33', 28.6), ('Pg34', 26),
        ('Pg35', 34.8), ('Pg36', 26.4), ('Pg37', 32), ('Pg38', 43), ('Pg39', 42)
    ]),
    OrderedDict([
        ('Pg30', H_G1), ('Pg31', 35), ('Pg32', 42), ('Pg33', 32), ('Pg34', 30),
        ('Pg35', 39), ('Pg36', 30), ('Pg37', 32), ('Pg38', 43), ('Pg39', 46)
    ])
]
N_H = len(H_values)

measure_exact = []
data_normalized = []
measure = []

var_names = [var_name.format(bus_ID) for bus_ID in np.unique(rec_bus_IDs) for var_name in var_names_fmt]
data_mean = {var_name: x_train_mean[k] for k,var_name in enumerate(var_names)}
data_std = {var_name: x_train_std[k] for k,var_name in enumerate(var_names)}

for H in H_values:
    data_file = data_dir + '/ieee39_' + '_'.join(map(lambda h: f'{h:.3f}', H.values())) + '.h5'
    _,_,v,_ = read_area_values(data_file, generators_areas_map['default'], P_nom, area_measure)
    measure_exact.append([v[area_ID - 1] for area_ID in area_IDs])

    t, _, data_norm, data_sliding, _ = load_data_slide([data_file],
                                                        var_names,
                                                        data_mean,
                                                        data_std,
                                                        window_dur,
                                                        window_step,
                                                        add_omega_ref = False,
                                                        verbose = True)
    data_normalized.append(data_norm)
    dt = np.diff(t[:2])[0]
    time, HH, _ = predict(model, data_sliding, window_step)
    measure.append(HH)
measure_exact = np.array(measure_exact)

In [ ]:
MAPE = np.array([[np.nanmean(np.abs((measure_exact[i,j] - measure[i][:,j]) / measure_exact[i,j])) * 100 \
                   for j in range(3)] for i in range(N_H)])
for i,row in enumerate(MAPE[1:]):
    print(f'{i+1} & ' + ' & '.join(map(lambda v: f'{v:.2f}\\%', row)) + ' \\\\')

In [ ]:
rows = 2
cols = 2
x_offset = 0.15
x_border = 0.02
x_space = 0.07
y_offset = 0.16
y_border = 0.05
y_space = 0.07
w = (1 - (x_offset + x_border + x_space * (cols - 1))) / cols
h = (1 - (y_offset + y_border + y_space * (rows - 1))) / rows

labels = 'ABCD'
fig = plt.figure(figsize=(figure_width/2.54, figure_height/2.54))
ax = [plt.axes([x_offset + j * (w + x_space), y_offset + i * (h + y_space), w, h]) \
     for i in range(rows-1, -1, -1) for j in range(cols)]
cmap = plt.get_cmap('gray', 4)
for i,a in enumerate(ax):
    for j in range(3):
        a.plot(time[[0,-1]] / 60, measure_exact[i+1,j] + np.zeros(2), '--', color=cmap(j), lw=1)
        a.plot(time / 60, measure[i+1][:,j], color=cmap(j), lw=1.5, label=f'Area {j+1}')
    for side in 'right','top':
        a.spines[side].set_visible(False)
    a.set_ylim([0.15, 0.47])
    a.set_xticks(np.r_[0 : 80 : 20])
    a.set_yticks(np.r_[0.2 : 0.5 : 0.1])
    if i > 1:
        a.set_xlabel('Time [min]', fontsize=8)
    else:
        a.set_xticklabels([])
    if i % 2 == 0:
        a.set_ylabel(f'{area_measure.capitalize()} [{measure_units}]', fontsize=8)
    else:
        a.set_yticklabels([])
    #a.grid(which='major', axis='y', ls=':', lw=0.5, color=[.6,.6,.6])
    a.tick_params(axis='x', labelsize=8)
    a.tick_params(axis='y', labelsize=8)
    a.text(-12, 0.47, labels[i], fontsize=10)
output_filename = f'IEEE39_areas{"-".join(map(str, area_IDs))}_H_G1={H_G1}_' + \
    f'rec_buses={rec_bus_list}_load_buses={stoch_load_bus_list}_const_H_{experiment_ID[:6]}.pdf'
fig.savefig(output_filename)

## Step of measure with transient

In [ ]:
data_file = data_dir + '/ieee39_4_steps.h5'

_,_,measure_exact,tstop = read_area_values(data_file, generators_areas_map['default'], P_nom, area_measure)
t, _, data_normalized, data_sliding, _ = load_data_slide([data_file],
                                                         var_names,
                                                         None,
                                                         data_std,
                                                         window_dur,
                                                         window_step,
                                                         add_omega_ref = False,
                                                         verbose = True)

dt = np.diff(t[:2])[0]
time, measure, _ = predict(model, data_sliding, window_step)

In [ ]:
N_blocks = tstop.size
fig = plt.figure(figsize=(figure_width/2.54, figure_height/2.54))

rows = 3
cols = 1
x_offset = 0.15
x_border = 0.02
x_space = 0.1
y_offset = 0.15
y_border = 0.02
y_space = 0.05
w = (1 - (x_offset + x_border + x_space * (cols - 1))) / cols
h = (1 - (y_offset + y_border + y_space)) / rows

ax = [
    plt.axes([x_offset, y_offset + (rows-1) * h + y_space, w, h]),
    plt.axes([x_offset, y_offset, w, h * (rows - 1)])
]

var_name = 'Vd_bus3'
ax[0].plot(t / 60, data_normalized[var_name], color='k', lw=0.5)
cmap = plt.get_cmap('gray', 4)
for j in range(3):
    for i in range(N_blocks):
        if i == 0:
            ax[1].plot([0, tstop[i]/60], measure_exact[j,i] + np.zeros(2), '--', color=cmap(j), lw=1)
        else:
            ax[1].plot([tstop[i-1]/60, tstop[i]/60], measure_exact[j,i] + np.zeros(2), '--', color=cmap(j), lw=1)
    ax[1].plot(time/60, measure[:,j], color=cmap(j), lw=1, label=f'Area {j+1}')
for a in ax:
    a.grid(which='major', axis='y', lw=0.5, ls=':')
    a.set_xlim([-2, 62])
    a.tick_params(axis='x', labelsize=8)
    a.tick_params(axis='y', labelsize=8)
    a.set_xticks(np.r_[0 : (N_blocks+1) * 15  : 15])
    for side in 'right','top':
        a.spines[side].set_visible(False)
ax[0].set_ylabel(r'V$_{d,3}$')
ax[0].set_xticklabels([])
ax[0].set_yticks(np.r_[-4 : 5 : 2])
ax[1].set_xlabel('Time [min]', fontsize=8)
ax[1].set_ylabel(f'{area_measure.capitalize()} [{measure_units}]', fontsize=8)
# ax[1].legend(loc='center left', fontsize=8)
ax[1].legend(loc='upper left', bbox_to_anchor=(0.015, 0.62), fontsize=8)
# fig.tight_layout()
output_filename = f'IEEE39_areas{"-".join(map(str, area_IDs))}_H_G1={H_G1}_' + \
    f'rec_buses={rec_bus_list}_load_buses={stoch_load_bus_list}_4_steps_H_{experiment_ID[:6]}.pdf'
fig.savefig(output_filename)

## Area overload

In [ ]:
LAMBDA = np.r_[0 : 0.1 : 0.01]
measure_exact = []
data = []
data_normalized = []
measure = []

var_names = [var_name.format(bus_ID) for bus_ID in np.unique(rec_bus_IDs) for var_name in var_names_fmt]
data_mean = {var_name: x_train_mean[k] for k,var_name in enumerate(var_names)}
data_std = {var_name: x_train_std[k] for k,var_name in enumerate(var_names)}

for l in LAMBDA:
    data_file = data_dir + '/ieee39_' + '_'.join(map(lambda h: f'{h:.3f}', default_H.values())) + \
        f'_lambda={l:.3f}.h5'
    _,_,v,_ = read_area_values(data_file, generators_areas_map['default'], P_nom, area_measure)
    measure_exact.append([v[area_ID - 1] for area_ID in area_IDs])

    t, dat, data_norm, data_sliding, _ = load_data_slide([data_file],
                                                        var_names,
                                                        None,
                                                        data_std,
                                                        window_dur,
                                                        window_step,
                                                        add_omega_ref = False,
                                                        verbose = True)
    data.append(dat)
    data_normalized.append(data_norm)
    dt = np.diff(t[:2])[0]
    time, HH, _ = predict(model, data_sliding, window_step)
    measure.append(HH)
measure_exact = np.array(measure_exact)

In [ ]:
fig = plt.figure(figsize=(figure_width/2.54, 8/2.54))

rows = 3
cols = 1
x_offset = 0.15
x_border = 0.02
x_space = 0.1
y_offset = 0.12
y_border = 0.3
y_space = 0.05
w = (1 - (x_offset + x_border + x_space * (cols - 1))) / cols
h = (1 - (y_offset + y_border + y_space * (rows - 1))) / rows

ax = [plt.axes([x_offset, y_offset + i * (h + y_space), w, h]) for i in range(rows-1, -1, -1)]

y_offset = y_offset + rows * (h + y_space) + 0.07
y_border = 0.02
x_space = 0.05
rows = 1
cols = 3
w = (1 - (x_offset + x_border + x_space * (cols - 1))) / cols
h = (1 - (y_offset + y_border + y_space * (rows - 1))) / rows

for i in range(cols):
    ax.append(plt.axes([x_offset + i * (w + x_space), y_offset, w, h]))

cmap = plt.get_cmap('gray', LAMBDA.size + 2)
lim = [
    [0.96, 1.01],
    [0.96, 1.02],
    [0.9, 1.7],
    [955, 975],
    [-4,4],
    [0,0.5]
]
ticks = [
    np.r_[0.97 : 1.01 : 0.015],
    np.r_[0.97 : 1.02 : 0.015],
    np.r_[1 : 1.7 : 0.2],
    np.r_[955 : 980 : 10],
    np.r_[-4 : 5 : 4],
    lim[-1]
]
for i in range(3):
    for j,l in enumerate(LAMBDA):
        ax[i].plot(time / 60, measure[j][:,i] / measure_exact[0,i], 
               color=cmap(j), lw=1, label=r'$\lambda={}$'.format(l))
    ax[i].set_xlim([-1, 61])
    ax[i].set_ylim(lim[i])
    ax[i].set_xticks(np.r_[0 : 70 : 20])
    if i != 2:
        ax[i].set_xticklabels([])
    ax[i].set_yticks(ticks[i])
    ax[i].grid(which='major', axis='y', lw=0.5, ls=':', color=[.6,.6,.6])

var_name = 'Vd_bus3'

for i in range(LAMBDA.size):
    n,edges = np.histogram(data[i][var_name], bins=101, density=True)
    ax[3].plot(edges[:-1] + np.diff(edges[:2])[0] / 2, n, color=cmap(i), lw=1)
    n,edges = np.histogram(data_normalized[i][var_name], bins=25, range=(-4,4), density=True)
    ax[4].plot(edges[:-1] + np.diff(edges[:2])[0] / 2, n, color=cmap(i), lw=1)
    f,Pxx = welch(data_normalized[i][var_name], 1/dt)
    ax[5].plot(f, Pxx, color=cmap(i), lw=1)

for i in range(3,6):
    ax[i].set_xlim(lim[i])
    ax[i].set_xticks(ticks[i])
    ax[i].set_yticks([])

ax[3].set_xlabel('Voltage [V]', fontsize=8)
ax[4].set_xlabel('Norm. V', fontsize=8)
ax[5].set_xlabel('Frequency [Hz]', fontsize=8)

ax[3].set_ylabel('Distribution', fontsize=8)
ax[5].set_ylabel('PSD', fontsize=8)

for a in ax:
    for side in 'right','top':
        a.spines[side].set_visible(False)
    a.tick_params(axis='x', labelsize=8)
    a.tick_params(axis='y', labelsize=8)

ax[1].set_ylabel(f'Normalized {area_measure}', fontsize=8)
ax[2].set_xlabel('Time [min]', fontsize=8)
output_filename = f'IEEE39_areas{"-".join(map(str, area_IDs))}_H_G1={H_G1}_' + \
    f'rec_buses={rec_bus_list}_load_buses={stoch_load_bus_list}_overload_{experiment_ID[:6]}.pdf'
fig.savefig(output_filename)

## Sinusoidal variation of the loads

In [ ]:
data_file = data_dir + '/ieee39_sin_loads_500.000_30.300_35.800_28.600_26.000_34.800_26.400_24.300_34.500_42.000.h5'

_,_,measure_exact,tstop = read_area_values(data_file, generators_areas_map['default'], P_nom, area_measure)
t, _, data_normalized, data_sliding, _ = load_data_slide([data_file],
                                                         var_names,
                                                         None,
                                                         data_std,
                                                         window_dur,
                                                         window_step,
                                                         add_omega_ref = False,
                                                         verbose = True)

dt = np.diff(t[:2])[0]
time, measure, _ = predict(model, data_sliding, window_step)

In [ ]:
fig,ax = plt.subplots(1, 1)
cmap = plt.get_cmap('gray',4)
for i in range(3):
    ax.plot(time[[0,-1]] / (60 * 60), measure_exact[i] + np.zeros(2), '--', lw=1, color=cmap(i))
    ax.plot(time / (60 * 60), measure[:,i], color=cmap(i), lw=1, label=f'Area {i+1}')
ax.legend(loc='center left')
for side in 'right','top':
    ax.spines[side].set_visible(False)
ax.set_xlabel('Time [hour]')
ax.set_ylabel(f'{area_measure.capitalize()} [{measure_units}]')
fig.tight_layout()

## With a 4th generator (a compensator) added in area 1

In [ ]:
P_nom['Pg99'] = 100e6

H_values = [
    default_H,
    OrderedDict([
        ('Pg30', H_G1), ('Pg31', 30.3), ('Pg32', 35.8), ('Pg33', 28.6), ('Pg34', 26),
        ('Pg35', 34.8), ('Pg36', 26.4), ('Pg37', 24.3), ('Pg38', 34.5), ('Pg39', 42),
        ('Pg99', 10)
    ]),
    OrderedDict([
        ('Pg30', H_G1), ('Pg31', 30.3), ('Pg32', 35.8), ('Pg33', 28.6), ('Pg34', 26),
        ('Pg35', 34.8), ('Pg36', 26.4), ('Pg37', 24.3), ('Pg38', 34.5), ('Pg39', 42),
        ('Pg99', 20)
    ]),
    OrderedDict([
        ('Pg30', H_G1), ('Pg31', 30.3), ('Pg32', 35.8), ('Pg33', 28.6), ('Pg34', 26),
        ('Pg35', 34.8), ('Pg36', 26.4), ('Pg37', 24.3), ('Pg38', 34.5), ('Pg39', 42),
        ('Pg99', 40)
    ])
]
N_H = len(H_values)

data_normalized = []
measure = []
measure_exact = []
for H in H_values:
    if 'Pg99' in H:
        data_file = data_dir + '/ieee39_compensator_' + '_'.join(map(lambda h: f'{h:.3f}', H.values())) + '.h5'
        _,_,v,_ = read_area_values(data_file, generators_areas_map['compensator'], P_nom, area_measure)
    else:
        data_file = data_dir + '/ieee39_' + '_'.join(map(lambda h: f'{h:.3f}', H.values())) + '.h5'
        _,_,v,_ = read_area_values(data_file, generators_areas_map['default'], P_nom, area_measure)

    measure_exact.append([v[area_ID - 1] for area_ID in area_IDs])

    t, _, data_norm, data_sliding, _ = load_data_slide([data_file],
                                                        var_names,
                                                        None,
                                                        data_std,
                                                        window_dur,
                                                        window_step,
                                                        add_omega_ref = False,
                                                        verbose = True)
    data_normalized.append(data_norm)
    dt = np.diff(t[:2])[0]
    time, pred, _ = predict(model, data_sliding, window_step)
    measure.append(pred)
measure_exact = np.array(measure_exact)

In [ ]:
measure_exact

In [ ]:
[np.nanmean(m, axis=0) for m in measure]

In [ ]:
fig,ax = plt.subplots(3, 1, sharex=True, figsize=(9,9))
cmap = plt.get_cmap('jet', N_H+1)
for i,a in enumerate(ax):
    for j in range(N_H):
        if i == 0:
            a.plot(time[[0,-1]] / 60, measure_exact[j,i] + np.zeros(2), '--', color=cmap(j), lw=1)
        elif j == 0:
            a.plot(time[[0,-1]] / 60, measure_exact[j,i] + np.zeros(2), '--', color='k', lw=1)
        a.plot(time / 60, measure[j][:,i], color=cmap(j), lw=1.5)
    for side in 'right','top':
        a.spines[side].set_visible(False)
    a.set_ylabel(f'Area {i+1}')
ax[-1].set_xlabel('Time [min]')
fig.tight_layout()